# ADM HW 3

For this homework, no dataset has been provided. Instead, you have to build your own. Your search engine will run on text documents. So, here we detail the procedure to follow for the data collection. We strongly suggest you work on different modules when implementing the required functions. For example, you may have a crawler.py module, a parser.py module, and a engine.py module: this is a good practice that improves readability in reporting and efficiency in deploying the code. Be careful; you are likely dealing with exceptions and other possible issues!

### 1.1. Get the list of Michelin restaurants
* You should begin by compiling a list of restaurants to include in your document corpus. Specifically, you will focus on web scraping the Michelin Restaurants in Italy. Your task is to collect the URL associated with each restaurant in this list. The output of this step should be a .txt file where each line contains a single restaurant’s URL. By the end, you should have approximately 2,000 restaurants on your list. The number changes daily, so some groups might have different number of restaurants.

---
We will start by loading the relevant libraries first. Then we will try to scrape the *relevant* links on only the first page of the [Michelin website](https://guide.michelin.com/en/it/restaurants/) to test. If the operation goes on successfully, we will scrape the links from the 100 pages!


In [1]:
# import the relevant libraries
import requests
from bs4 import BeautifulSoup
import os
import csv
import time
from tqdm import tqdm
import asyncio
from scripts.crawler import *
from scripts.myparser import *
from scripts.functions import *
import pandas as pd

[nltk_data] Downloading package punkt to /Users/saifdev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saifdev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/saifdev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# setting current working directory to /Users/saifdev/Desktop/ADM/ADM_HW3
os.chdir("/Users/saifdev/Desktop/ADM/ADM_HW_3")

Before I begin to build the function `scraping_urls`, which will scrape all the links for the Michelin restaurants in Italy and save them to `urls.txt`, I will start by trying to scrape only the first page to test and practice.

In [ ]:
# A SMALL TEST
url = "https://guide.michelin.com/en/it/restaurants/page/" # url of the Michelin Guide Italy
with open('Files/test.txt', 'w') as file: # open a file to write the links of the restaurants
    current_page = url + "1" # the first page only for testing
    request = requests.get(current_page) # get the first page
    soup = BeautifulSoup(request.content, 'lxml') # parse the content of the first page
    for a in soup.select("div.flex-fill a"): # Ref Keith Galli Y.T. channel :)
        file.write("https://guide.michelin.com"+a.get('href') + '\n')   # write the links of the restaurants in the first page

In [ ]:
# check the number of lines written in the file = 20 (and also manually checked)
with open('Files/test.txt', 'r') as file: # open the file to read the links of the restaurants
    num_lines = len(file.readlines()) # count the number of lines in the file
    result = "20" if num_lines == 20 else "not 20" # check if the number of lines is 20
    print(f"The number of urls scraped is {result}") # print the result. It is indeed 20


The number of urls scraped is 20


It worked as expected. Now we will scrape the urls from the whole 100 pages on the Micheline website using the `scraping_urls` function.

A little note about the `scraping_urls` function:
* It is a function that takes the url of the Michelin Guide Italy webpage and the number of pages to scrape as input and returns a text file containing the URLs of all the restaurants. The function is defined in the crawler.py file.

In [ ]:
pages = 100 # set the number of pages to scrape
url = "https://guide.michelin.com/en/it/restaurants/page/" # url of the Michelin Guide Italy
scraping_urls(url=url, pages=pages) # scrape the urls of the restaurants in the Michelin Guide Italy

Scraping Pages:   0%|          | 0/100 [00:00<?, ?it/s]

Checking if the number of urls scraped is approximately 2000

In [ ]:
# check the number of urls scraped
with open('Files/urls.txt', 'r') as file:
    num_lines = len(file.readlines())
    print(f"Total Number of restaurants urls scraped is {num_lines}")


Total Number of restaurants urls scraped is 2027


---

### 1.2. Crawl Michelin restaurant pages

##### Once you have all the URLs on the list, you should:

* Download the HTML corresponding to each of the collected URLs.
* After collecting each page, immediately save its HTML in a file. This way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
* Organize the downloaded HTML pages into folders. Each folder will contain the HTML of the restaurants from page 1, page 2, ... of the Michelin restaurant list.
* Tip: Due to the large number of pages to download, consider using methods that can help shorten the process. If you employed a particular process or approach, kindly describe it.

---

Now, we will scrape the information of the restaurants from the urls scraped. To do this, we will use the function load_urls, fetch_and_save, and download_html_in_batches. All these functions are defined in the crawler.py file.

Some information about each of these functions:
* **load_urls**: This function loads the urls of the restaurants from the urls.txt file.
* **fetch_and_save**: This function fetches the html content of the urls and saves the html file into a folder of a particular batch/page.
* **download_html_in_batches**: This function downloads the html content of the urls in batches of 20. The function makes used of `asyncio` and `aiohttp` to download the html content in parallel. This function is used to speed up the process of downloading the html content of the urls due to concurrency.

In [ ]:
url_file = 'Files/urls.txt' # file containing the urls of the restaurants
output_dir = 'michelin_html_batches' # directory to save the sub-directories and the html files
urls = load_urls(url_file) # load the urls of the restaurants

start_time = time.time()
try:
    await download_html_in_batches(urls, output_dir) # using asyncio environment to download the html files in batches
except RuntimeError:
    # Check if there's already an event loop running. Why? Because we can't create a new event loop in a thread that already has
    # an event loop running. Jupyter notebooks often have an event loop running by default. If we try to create a new event loop
    # in a Jupyter notebook, we'll get a RuntimeError. Thus, use asyncio.run if not in a running event loop environment
    asyncio.run(download_html_in_batches(urls, output_dir))

print(f"Finished downloading in {time.time() - start_time} seconds")

Now, we have the html files of the restaurants in the Michelin Guide Italy website downloaded in batches - organised into folders. Now is the time that we start parsing them.

---
### 1.3 Parse downloaded pages

##### At this point, you should have all the HTML documents about the restaurant of interest, and you can start to extract specific information. The list of the information we desire for each restaurant and their format is as follows:

* Restaurant Name (to save as restaurantName): string;
* Address (to save as address): string;
* City (to save as city): string;
* Postal Code (to save as postalCode): string;
* Country (to save as country): string;
* Price Range (to save as priceRange): string;
* Cuisine Type (to save as cuisineType): string;
* Description (to save as description): string;
* Facilities and Services (to save as facilitiesServices): list of strings;
* Accepted Credit Cards (to save as creditCards): list of strings;
* Phone Number (to save as phoneNumber): string;
* URL to the Restaurant Page (to save as website): string. 


---

we have created many functions to parse the html files and extract the information we need. These functions ae in the myparser.py file. Here is the list of functions. The functions are listed below and their names are self-explanatory:  

* `get_restaurant_name(soup)` - taking in soup object and returning the name of the restaurant for example.
* `get_address(soup)`
* `get_city(address)`
* `get_postal_code(address)`
* `get_country(address)`
* `get_price_range(soup)`
* `get_cuisine_type(soup)`
* `get_description(soup)`
* `get_facilities_services(soup)`
* `get_credit_cards(soup)`
* `get_phone_number(soup)`
* `get_website(soup)`

We will test these functions of a random html file form the directories we have created. This will help us to understand the output of these functions and help us evaluate how well they perform.

In [ ]:
# Test the parser functions on a particular html file
with open('/Users/saifdev/Desktop/ADM/ADM_HW_3/michelin_html_batches/batch_1/20tre.html', 'r', encoding='utf-8') as file:
	content = file.read()
	soup = BeautifulSoup(content, 'lxml')
	name = get_restaurant_name(soup)
	print(f"Restaurant name: {name}")
	address = get_address(soup)
	print(f"Restaurant address: {address}")
	city = get_city(address)
	print(f"Restaurant city: {city}")
	zipcode = get_postal_code(address)
	print(f"Restaurant zipcode: {zipcode}")
	country = get_country(address)
	print(f"Restaurant country: {country}")
	price = get_price_range(soup)
	print(f"Restaurant price range: {price}")
	cuisine = get_cuisine_type(soup)
	print(f"Restaurant cuisine type: {cuisine}")
	description = get_description(soup)
	print(f"Restaurant description: {description}")
	facilities = get_facilities_services(soup)
	print(f"Restaurant facilities and services: {facilities}")
	credit_cards = get_credit_cards(soup)
	print(f"Restaurant credit cards: {credit_cards}")
	phone = get_phone_number(soup)
	print(f"Restaurant phone number: {phone}")
	website = get_website(soup)
	print(f"Restaurant website: {website}")

Restaurant name: 20Tre
Restaurant address: via David Chiossone 20 r, Genoa, 16123, Italy
Restaurant city: Genoa
Restaurant zipcode: 16123
Restaurant country: Italy
Restaurant price range: €€
Restaurant cuisine type: Farm to table, Modern Cuisine
Restaurant description: Situated in the heart of Genoa’s historic centre, this contemporary-style restaurant focuses on just a few dishes, almost all fish-based, presented in a very modern style and in generous portions. Seasonal ingredients and market-fresh produce are the guiding philosophy here.
Restaurant facilities and services: ['Air conditioning']
Restaurant credit cards: ['amex', 'dinersclub', 'mastercard', 'visa']
Restaurant phone number: +39 010 247 6191
Restaurant website: https://www.ristorante20tregenova.it/


Our Parser functions work perfectly. Let's employ them now to get the required data from all the html files we have stored. Then, we will gather the data into a csv file which we can later read into the Pandas DataFrame.

To do this, we will use the following functions stored in `myparser.py`:
- `extract_restaurant_data`, `save_restaurant_data_to_tsv`, `forming_a_tsv`

A brief explanation of the functions:
* `extract_restaurant_data` - This function takes the html content of a restaurant page and extracts the relevant data using the parser functions.
* `save_restaurant_data_to_tsv` - This function takes the extracted data and saves it to a tsv file.
* `forming_a_tsv` - This function takes the html files stored in the directory and extracts the data from each file and saves it to a tsv file.


In [ ]:
# form a csv file with the information of the restaurants using extract_restaurant_data function and the save_restaurant_data_to_csv function
forming_a_tsv()

Data saved to Files/michelin_restaurant_data.tsv


Finally, we have the csv file with the information of the restaurants using the aforementioned functions. The csv file is named *"michelin_restaurants.csv"*. We can now load the csv file using `pd.read_csv` and perform the required tasks.

In [3]:
data = pd.read_csv('Files/michelin_restaurant_data.tsv', sep='\t')
data.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Locanda Radici,"SP 21, contrada San Vincenzo, Melizzano, 82030...",Melizzano,82030.0,Italy,€€,"Modern Cuisine, Campanian",A rustic restaurant with contemporary Mediterr...,Air conditioning; Car park; Garden or park; Gr...,amex; mastercard; visa,+39 0824 944506,https://www.locandaradici.it/
1,Posta,"viale Vittorio Veneto 169, Sant'Omobono Terme,...",Sant'Omobono Terme,24038.0,Italy,€€€,Italian,"Situated in the Imagna valley, this welcoming,...",Air conditioning; Wheelchair access,amex; dinersclub; mastercard; visa,+39 035 851134,https://www.frosioristoranti.it
2,Hostaria Baccofurore,"via G.B. Lama 9, Furore, 84010, Italy",Furore,84010.0,Italy,€€,"Regional Cuisine, Farm to table",Patience is needed to get to this restaurant f...,Air conditioning; Car park; Garden or park; Gr...,amex; dinersclub; jcb; maestrocard; mastercard...,+39 089 830360,https://www.baccofurore.it/
3,Nni Lausta,"via Risorgimento 188, Santa Marina Salina, 980...",Santa Marina Salina,98050.0,Italy,€€,"Seafood, Seasonal Cuisine","Fish plays the starring role here, where the a...",Terrace,amex; jcb; maestrocard; mastercard; visa,+39 090 984 3486,NaN
4,Osteria de Börg,"via Forzieri 12, Rimini, 47921, Italy",Rimini,47921.0,Italy,€,"Cuisine from Romagna, Traditional Cuisine",Borgo San Giuliano is situated just a stroll f...,Terrace,amex; maestrocard; mastercard; visa,+39 0541 56074,https://www.osteriadeborg.it/


In [4]:
print(f"Checking the shape of the data: It should be {data.shape}")

Checking the shape of the data: It should be (2029, 12)


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   restaurantName      2029 non-null   object 
 1   address             2029 non-null   object 
 2   city                2000 non-null   object 
 3   postalCode          1983 non-null   float64
 4   country             2029 non-null   object 
 5   priceRange          1983 non-null   object 
 6   cuisineType         2029 non-null   object 
 7   description         2029 non-null   object 
 8   facilitiesServices  1959 non-null   object 
 9   creditCards         1979 non-null   object 
 10  phoneNumber         1983 non-null   object 
 11  website             1875 non-null   object 
dtypes: float64(1), object(11)
memory usage: 190.3+ KB


To do the preprocessing of the data, first we need to check if there exists missing or empty values in the data.



In [6]:
missing_values = data.isnull().sum()
print(f"Missing values in the data: \n{missing_values}")
print(f"The total number of empty descriptions are: {data[data['description'] == ''].shape[0]}")

Missing values in the data: 
restaurantName          0
address                 0
city                   29
postalCode             46
country                 0
priceRange             46
cuisineType             0
description             0
facilitiesServices     70
creditCards            50
phoneNumber            46
website               154
dtype: int64
The total number of empty descriptions are: 0


In [7]:
# filling the missing values of columns cuisineType and facilitiesServices with empty strings. This will be useful for the search engine later
data['cuisineType'] = data['cuisineType'].fillna('')
data['facilitiesServices'] = data['facilitiesServices'].fillna('')

Because we have no missing values or emtpy values in the description column, we will, for now, not remove any rows where some of the data is missing.

# 2.0 Preprocessing
## 2.0.0) Preprocessing the text

Before building the search engine, we must clean and prepare the text in each restaurant’s description.
To do this we implemented the  `preprocessing` (defined in `functions.py`):

We will:

+ Remove stopwords.
+ Remove punctuation.
+ Bring everything to lowercase.
+ Handle compund words with '-'.
+ Apply stemming.
>For this, we use the nltk library.

We will apply the preprocessing function on every row of the 'description' column of our dataset, to then create a 'cleaned_description' column in which each row contains the tokenized and ready-to-use version of the documents.


We will apply the preprocessing function on every row of the 'description' column of our dataset, to then create a 'cleaned_description' column in which each row contains the tokenized and ready-to-use version of the documents.

In [8]:
data['cleaned_description'] = data['description'].apply(preprocessing)

In [9]:
data['cleaned_description'].head()

0    [rustic, restaur, contemporari, mediterranean,...
1    [situat, imagna, valley, welcom, famili, run, ...
2    [patienc, need, get, restaur, coast, result, w...
3    [fish, play, star, role, authent, tradit, flav...
4    [borgo, san, giuliano, situat, stroll, sea, ol...
Name: cleaned_description, dtype: object

In [10]:
data.to_csv('michelin_restaurant_data.csv', index=False)

## 2.1 Conjunctive Query
This first version of the search engine narrows the search to the description field of each restaurant. Only restaurants whose descriptions contain **all** the query words will be returned.
### 2.1.1 Creating our Index! 
>Vocabulary File: Create a file called vocabulary.csv that maps each word to a unique integer (term_id).

The function `create_vocabulary` from our `functions.py`  builds a vocabulary from the input descriptions of each restaurant of our dataset by assigning a unique term ID to each distinct term.


In [11]:
vocabulary = create_vocabulary(data['cleaned_description'])

In [12]:
with open('Files/vocabulary.csv', mode='r') as file:
    reader = csv.reader(file)
    for i, row in enumerate(reader):
        if i == 0:
            continue  # Skip the header row
        if i > 10:  # Print first 10 rows
            break
        print(row)
print(f"Our vocabulary contains {len(vocabulary)} unique words. This may vary according to description preprocessing choices")

['top', '1']
['owner', '2']
['work', '3']
['greeneri', '4']
['window', '5']
['made', '6']
['surround', '7']
['mediterranean', '8']
['ingredi', '9']
['modern', '10']
Our vocabulary contains 7853 unique words. This may vary according to description preprocessing choices


>Inverted Index: Build a dictionary mapping each term_id to a list of document IDs where that term appears.

The function `create_inverted_index` builds a vocabulary from the input descriptions by assigning a unique term ID to each distinct term. Then it writes the resulting vocabulary to a CSV file.

Steps:

1. Initializes an empty vocabulary dictionary and term IDs starting at 1.
2. Iterates through each description and extracts unique words (using a set to avoid duplicates).
3. Adds terms to the vocabulary if they are digits or have more than one character.
4. Saves the vocabulary to a CSV file named 'vocabulary.csv', with columns 'term' and 'term_id'.


In [13]:
inverted_index = create_inverted_index(data['cleaned_description'], vocabulary)

### 2.1.2 Execute the Query
When the user inputs a query, for example, "vegan restaurant with garden", the search engine will:

- Preprocess the string containing the search query using the `preprocessing` function.
- For each unique term in the query, its term ID is fetched from the dictionary, that is the `vocabulary.csv` file.
- If the term ID exists, the corresponding document list is retrieved from the `inverted_index.json` file and stored.
- Iteratively compute the intersection of document lists to find documents containing all query terms.
- Retrieve and return matching document details (restaurantName, address, description, website) from the dataset.


The function outputs a DataFrame with details of matching documents or None if no matches are found.

In [14]:
def conjuntive_query(query, dictionary, inverted_index):

    #preprocessing query text with the same function used to clean descriptions
    cleaned_query = preprocessing(query)

    unique_words = set(cleaned_query)

    doc_lists = []
    for word in unique_words:
        #retrive the corresponding term_id from the dictionary
        term_id = dictionary.get(word)
        if term_id:
            #retrive and append the list of documents containing term_id
            docs = inverted_index.get(str(term_id))
            if list:
                doc_lists.append(docs)
    
    if doc_lists:
        intersection = doc_lists[0] #initialize as the first list
        for docs in doc_lists[1:]:
            intersection = list(set(intersection) & set(docs)) #iteratively intersect with any other list
    else:
        intersection = []

    result = data.loc[intersection, ['restaurantName', 'address', 'description', 'website']]
    print(query)

    if intersection:
        return result
    else:
        print('No matches found')
        return None

Now we will test the search engine on the query: *'Vegan restaurant with garden'* after loading both the dictionary and the inverted index using `load_dictionary` and `load_inverted_index` functions.

In [15]:
dictionary = load_dictionary('Files/vocabulary.csv')
inverted_index = load_inverted_index('Files/inverted_index.json')
conjuntive_query(input('Input your query'), dictionary,inverted_index)

modern seasonal cuisine


,restaurantName,address,description,website
256,Pipero Roma,"corso Vittorio Emanuele II 250, Rome, 00186, I...",Situated opposite the church of Santa Maria in...,https://www.piperoroma.it/
514,Osteria Ophis,"corso Serpente Aureo 54/b, Offida, 63073, Italy",Situated in the beautiful historic centre of O...,https://www.osteriaophis.com/
645,Il Tino,"via Monte Cadria 127, Fiumicino, 00054, Italy",Enjoying an attractive location in the Nautilu...,https://www.ristoranteiltino.com/
1927,Flurin,"Laubengasse 2, Glorenza, 39020, Italy",Flurin occupies an old medieval tower in Glore...,https://www.flurin.it
1683,Aprudia,"largo del Forno 16, Giulianova, 64021, Italy","At this restaurant in the historic centre, whe...",http://www.aprudia.com
1811,Locanda Solagna,"piazza I Novembre 2, Quero, 32038, Italy",Although this restaurant has been in business ...,https://www.locandasolagna.it/
1049,San Giorgio,"viale Brigate Bisagno 69r, Genoa, 16129, Italy",Situated in the city albeit not right in the c...,https://www.ristorantesangiorgiogenova.it/
1818,Osteria del Miglio 2.10,"via Patrioti 2, Pieve San Giacomo, 26035, Italy",Although the town may not be of major importan...,NaN
1956,Le Vie del Borgo,"via alla Piazza 6, Toceno, 28858, Italy",Le Vie del Borgo is situated in a restored rus...,https://www.leviedelborgoguesthouse.it/
1447,Gallery Bistrot Contemporaneo,"via Regina Margherita 3/b, Troia, 71029, Italy","Modern, tasty and carefully curated cuisine, w...",NaN


We can visually check the correctness of the `conjuntive_query` search engine using an auxiliary tool defined as `highlight_matches` in `functions.py`: such function highlights (** **) query matches in the description field of the search results in the printed relevant restaurant descriptions. 


We will test the search engine for two queries: *'Vegan restaurant with garden'* and *'Romantic restaurant with terrace'.*
and check its correctness thanks to the `highlight_matches` function.

In [16]:
query = "vegan restaurant with garden"
result = conjuntive_query(query, dictionary, inverted_index)

if result is not None:
    query_words = preprocessing(query)  # Process query the same way as descriptions
    highlight_matches(result, query_words)
else:
    print("No matches found.")

vegan restaurant with garden
Restaurant: Il Mirto
Address: via Provinciale Lacco, 284, Forio, 80075, Italy
Description: When you arrive at the luxury Botania hotel, you’ll be accompanied through 3ha grounds of lush Mediterranean vegetation to the **restaur**ant. You couldn’t have a better introduction to the cuisine that awaits at Il Mirto, where you can choose between two long tasting menus or a shorter four-course option featuring dishes from these menus. One of the menus is vegetarian, the other **vegan** and almost all the vegetables are sourced from the property’s own kitchen **garden** or from other local growers from Ischia. A delightful experience in a beautiful natural setting, with excellent service provided by young, friendly staff.
Website: http://www.ilmirtoristorantevegetariano.it

Restaurant: Li Lioni
Address: SS 131 km 244,400 - regione Li Lioni, Porto Torres, 07046, Italy
Description: The highlights of this **restaur**ant just outside the tourist centre of Porto Torres

In [17]:
query = "romantic restaurant with terrace"
result = conjuntive_query(query, dictionary, inverted_index)

if result is not None:
    query_words = preprocessing(query)  # Process query the same way as descriptions
    highlight_matches(result, query_words)
else:
    print("No matches found.")

romantic restaurant with terrace
Restaurant: Urubamba
Address: via Gaetano Filangieri 16/c, Naples, 80121, Italy
Description: Situated amid luxury boutiques and historic palazzi in one of Naples’ most exclusive districts, this **restaur**ant boasts a chic and **romant**ic dining room with soft lighting on the first floor, as well as an exclusive outdoor **terrac**e for fine weather. The cuisine is totally in keeping with the trendy setting, featuring a successful combination of two different gastronomic cultures (namely Peruvian and Japanese) that blend together perfectly.
Website: http://www.urubamba.it

Restaurant: Sogno
Address: via Porto San Felice 41, San Felice del Benaco, 25010, Italy
Description: This aptly named **restaur**ant (“sogno” is the Italian for dream) really is a dream come true, especially in summer when you can dine on the **romant**ic lakeside **terrac**e. The delicious Mediterranean-style cuisine will more than satisfy your tastebuds, while there’s also the optio

## 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity
For the second search engine, given a query, retrieve the top-k restaurants ranked by relevance to the query.

### 2.2.1 Inverted Index with TF-IDF Scores
> tfIdf Scores: Calculate TF-IDF scores for each term in each restaurant’s description.

Assume that: 

   + $q$ is our query;
   
   + $tf_{t,d}$ is the term frequency (number of occurrences of term t in document d);

   + $idf_{t}$ is the  inverse document frequency of term t defined by $idf_{t} = log(\frac{N}{df_{t}})$;
    
   + $N$ is the number of documents (restaurants);
    
   + $df_t$ is the the document frequency.

Then the tf-idf weighting scheme assigned to term $t$ is a weight in document d given by $$tf-idf_{t,d} = tf_{t,d} × idf_{t}$$

This allows to define the TF-IDF scores as the sum over all the terms of the query of the weights: $$ Score(q, d) = \sum_{t∈q} tf-idf_{t,d}$$

    
> Updated Inverted Index: Build a new inverted index where each entry is a term, and the value is a list of tuples containing document IDs and TF-IDF scores.




#### `tf_idf_inverted_index` 

This function generates a TF-IDF inverted index for a given dataset of cleaned restaurant descriptions.
It calculates the term frequency (TF) and inverse document frequency (IDF) for each term and stores the results in an inverted index format.

Steps:
  1. Calculate the document frequency (DF) for each term in the vocabulary.
  2. Calculate the term frequency (TF) for each document's cleaned description.
  3. Calculate the TF-IDF score for each term in each document.
  4. Store the results in an inverted index, where each term (from the vocabulary) is mapped to a 
       list of document IDs and their corresponding TF-IDF scores.
  5. Save the resulting inverted index as a JSON file ('tf_idf_inverted_index.json').

In [18]:
vocabulary = load_dictionary('Files/vocabulary.csv')
tf_idf_inverted_index = tf_idf_inverted_index(data, vocabulary)

### 2.2.2 Execute the Ranked Query
For the **ranked search engine**:

+ Process the query terms.
   > using the preprocessing function
+ Use Cosine Similarity to rank matching restaurants based on the TF-IDF vectors of the query and each document.
   > computing query vector and document vectors with one component in the vector for each dictionary term. Such components are computed using the tf-idf scores.
+ Return the top-k results or all matching restaurants if fewer than k have non-zero similarity.
+ Each result should include: restaurantName, address, description, website, Similarity score (between 0 and 1)

In [19]:
N = len(data)
N

2029

#### `query_vector` 
This function generates a TF-IDF vector for a search query based on a given vocabulary and inverted index, representing the query in the same space as document vectors for similarity comparisons, for ranking search results.

Input: 
- query: A string containing the search query.
- vocabulary: A dictionary mapping terms to unique IDs.
- inverted_index: A dictionary mapping term IDs to a list of document IDs and their TF-IDF scores.

Steps:
- The query is preprocessed (e.g., cleaned and tokenized).
- For each query term:
    + Check Existence: If the term is not in the vocabulary, it is skipped.
    + TF Calculation: Term frequency (TF) is computed based on the query.
    + IDF Calculation: Inverse document frequency (IDF) is computed using the document frequency (DF) from the inverted index.
    + TF-IDF Computation: Combines TF and IDF to calculate the TF-IDF score for the term.
    + Populate Vector: The TF-IDF score is stored in the query vector using the term’s ID.
- Final Vector: Builds the final vector, ensuring all terms in the vocabulary are represented (defaulting to zero if absent).

Output:

Returns a vector where each position corresponds to a term in the vocabulary and contains its TF-IDF score for the query.

In [20]:
vocabulary = load_dictionary('Files/vocabulary.csv')
inverted_index = load_inverted_index('Files/inverted_index.json')
v= query_vector('vegan restaurant with garden', vocabulary, inverted_index, data)

Term: vegan, TF: 1
Term: vegan, IDF: 7.615298339825815
Term: vegan, TF-IDF: 7.615298339825815
Term: restaur, TF: 1
Term: restaur, IDF: 7.615298339825815
Term: restaur, TF-IDF: 7.615298339825815
Term: garden, TF: 1
Term: garden, IDF: 7.615298339825815
Term: garden, TF-IDF: 7.615298339825815
Final Query Vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#### `doc_vectors` 
This function creates a document-term matrix $D$, where rows represent terms in the vocabulary, columns represent documents, and each entry contains the TF-IDF score of a term in a document.

Input:
- data: Dataset of documents.
- vocabulary: Maps terms to unique IDs.
- tf_idf_inverted_index: Maps term IDs to lists of document IDs and TF-IDF scores.

Matrix Initialization: 
- $D$ is a zero matrix with dimensions (num_terms,num_docs).

Matrix Population:

- For each term ID in the inverted index:
    + Adjust term and document indices to match matrix bounds.
    + Populate $D$ with TF-IDF scores for documents containing the term.

Output:
Returns the completed $D$, capturing the TF-IDF relationship between terms and documents.

In [21]:
D = doc_vectors(data, vocabulary, tf_idf_inverted_index)
D.shape

(7853, 2029)

The matrix $D$ represents the structure of the document vectors. Each column corresponds to a document, and each row represents a term from the vocabulary, identified by its term_id. The value at position $D[i,j]$ is the TF-IDF score of term $i$ in document $j$. Since the vocabulary contains 7853 terms, each document vector has 7853 components, reflecting the TF-IDF scores for all terms in the vocabulary. Our dataset represents 2029 documents (restaurants), so $D\in\mathbb{R}^{7853 x 2029}$.

Having defined both the query and the documents vectors, we can pairwise compute the **cosine similarity** using the implementation of the `cosine_similarity` function in `functions.py` file.

#### `ranked_search`
This function implements a **ranked search** engine to retrieve the top $k$ restaurants, based on their similarity to a user's query.

It takes in imput the user's search query and the number $k$ of top results to return.

It converts the query into a vector using the `query_vector` function, which maps the query terms to the vocabulary and computes their weights using the inverted index.
Then for each document in the dataset, it computes its cosine similarity with the query vector calling the `cosine_similarity` function and appends the resulting similarity score to an initialized list.
The similarity scores are finally added to the dataset as a new column (SimilarityScore) in order to sort the dataset by such scores in descending order to select the top $k$ results.

Te function outputs a DataFrame containing the top $k$ restaurants, along with their name, address, description, website, and similarity score.

In [22]:
ranked_search('vegan restaurant with garden',  3, vocabulary, data, inverted_index, D )

Term: vegan, TF: 1
Term: vegan, IDF: 7.615298339825815
Term: vegan, TF-IDF: 7.615298339825815
Term: restaur, TF: 1
Term: restaur, IDF: 7.615298339825815
Term: restaur, TF-IDF: 7.615298339825815
Term: garden, TF: 1
Term: garden, IDF: 7.615298339825815
Term: garden, TF-IDF: 7.615298339825815
Final Query Vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

,restaurantName,address,description,website,SimilarityScore
94,Sale Grosso,"vicolo De' Facchini 4/a, Bologna, 40124, Italy","A simple, bistro - style restaurant serving we...",NaN,0.264720
1245,Li Lioni,"SS 131 km 244,400 - regione Li Lioni, Porto To...",The highlights of this restaurant just outside...,https://www.tenutalilioni.it/,0.183576
194,Il Mirto,"via Provinciale Lacco, 284, Forio, 80075, Italy","When you arrive at the luxury Botania hotel, y...",http://www.ilmirtoristorantevegetariano.it,0.157729


In [23]:
ranked_search('romantic restaurant with terrace',  3, vocabulary, data, inverted_index, D )

Term: romant, TF: 1
Term: romant, IDF: 7.615298339825815
Term: romant, TF-IDF: 7.615298339825815
Term: restaur, TF: 1
Term: restaur, IDF: 7.615298339825815
Term: restaur, TF-IDF: 7.615298339825815
Term: terrac, TF: 1
Term: terrac, IDF: 7.615298339825815
Term: terrac, TF-IDF: 7.615298339825815
Final Query Vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

,restaurantName,address,description,website,SimilarityScore
1229,Capo Santa Chiara,"via al Capo di Santa Chiara 69, Boccadasse, Ge...","The chef at this modern, renovated restaurant ...",https://ristorantecaposantachiara.com/,0.214436
1884,Andrea Monesi - Locanda di Orta,"via Olina 18, Orta San Giulio, 28016, Italy",Situated in the historic centre of one of the ...,https://www.andreamonesi.com/,0.202419
1379,Villa Baroni,"via Acquadro 12, Bodio Lomnago, 21020, Italy",This romantic restaurant on the lakeshore has ...,https://www.villabaroni.it/,0.184400


In [24]:
conjuntive_query('romantic restaurant with terrace', vocabulary, inverted_index)

romantic restaurant with terrace


,restaurantName,address,description,website
513,Urubamba,"via Gaetano Filangieri 16/c, Naples, 80121, Italy",Situated amid luxury boutiques and historic pa...,http://www.urubamba.it
1283,Sogno,"via Porto San Felice 41, San Felice del Benaco...",This aptly named restaurant (“sogno” is the It...,https://www.sognogarda.it/
1936,"Il Giardino ""Da Felicin""","via Vallada 18, Monforte d'Alba, 12065, Italy","A bastion of classic Piedmontese cuisine, this...",https://www.felicin.it/
923,La Speranzina Restaurant & Relais,"via Dante 16, Sirmione, 25019, Italy","A romantic, Mediterranean-style restaurant in ...",https://www.lasperanzina.it/
1313,La Serra,"via Marconi 169, località Belvedere Fornillo, ...","Despite its bright, elegant and airy feel, the...",https://www.leagavi.it/ristorante-la-serra
1442,Al Monastero,"via Ristori 9, Cividale del Friuli, 33043, Italy",Enjoy excellent local hams and salami and othe...,https://www.almonastero.com/
167,Aria,"Via Costa 18, Lavagna, 16033, Italy",The Villa Riviera resort home to this restaura...,https://www.villarivieraresort.com/it/hotel-ri...
1080,Unterwirt,"Gudon 45, Gudon, 39043, Italy","Occupying a private house, this restaurant boa...",https://www.unterwirtgufidaun.com/it
827,Il Flauto di Pan,"via Santa Chiara 26, Ravello, 84010, Italy",It would be no exaggeration to say that this r...,https://www.hotelvillacimbrone.com/it/il-flaut...
444,Maxi,"via Luigi Serio 8, SS 145 Sorrentina km 14,500...","From the entrance to the Capo la Gala hotel, y...",https://www.hotelcapolagala.com/it/dining/maxi...


## 3. Define a New Score!
Now, we will define a custom ranking metric to prioritize restaurants based on user queries.

### Steps:
User query:

The user provides a text query. We’ll retrieve relevant documents using the search engine built in Step 2.1.
New Ranking Metric: After retrieving relevant documents, we’ll rank them using a new custom score. Instead of limiting the scoring to only the description field, we can include other attributes like priceRange, facilitiesServices, and cuisineType.
You will use a heap data structure (e.g., Python’s heapq library) to maintain the top-k restaurants.

### New Scoring Function:
Define a scoring function that takes into account various attributes:

Description Match: Give weight based on the query similarity to the description (using TF-IDF scores).
Cuisine Match: Increase the score for matching cuisine types.
Facilities and Services: Give more points for matching facilities/services (e.g., “Terrace,” “Air conditioning”).
Price Range: Higher scores could be given to more affordable options based on the user’s choice.

### Output:
The output should include:

restaurantName,
address,
description,
website,
The new similarity score based on the custom metric.
Are the results you obtain better than with the previous scoring function? Explain and compare results.


# Imports
### Introduction

In this cell, we import the necessary packages and load the files that will be used throughout the project, including:
- The TF-IDF inverted index file,
- The Michelin restaurant data,
- The vocabulary and inverted index for text processing.


In [25]:

with open('Files/tf_idf_inverted_index.json', 'r') as file:
    TFiDF = json.load(file)  # Load the TF-IDF inverted index

# Add a unique identifier column based on the description index
data['id'] = list(data['description'].keys())

# Load vocabulary from a CSV file
vocabulary = pd.read_csv('Files/vocabulary.csv')
vocabulary= vocabulary.set_index('term')['term_id'].to_dict()

# Load the inverted index from another JSON file
with open('Files/inverted_index.json', 'r') as file:
    inverted_index = json.load(file)  # Load the inverted index

# Functions

### Query Functions Explanation

This script defines two functions for handling restaurant queries:

### 1. conjunctive_query (query, dictionary, inverted_index)
This function processes a conjunctive query (AND search) to find restaurants that match all the keywords in the query as point 2.1.

- **Preprocessing**: The query is cleaned and tokenized into unique words.
- **Inverted Index Lookup**: For each unique word, the function retrieves the corresponding document IDs (restaurants) from the inverted index.
- **Intersection**: It computes the intersection of the document lists for all query words to find the common documents (restaurants).
- **Return Data**: The matching restaurants are then retrieved from the dataset using the intersection of document IDs and returned as a DataFrame with key restaurant details such as name, address, and description.

### 2. customized_query (query_text, dictionary, inverted_index, TFiDF, vocabulary, query_price, query_services)
This function refines the results from "conjunctive_query" by considering additional filters like price range, available services, and cuisine types.

- **Precise Match Filtering**: First, the "conjunctive_query" function is used to filter the restaurants based on the query text.
- **TF-IDF Scoring**: A TF-IDF score is calculated for each restaurant based on the query words. The score is accumulated for each matching term in the restaurant descriptions.
- **Price Range Scoring**: A price score is added, where restaurants with a price range close to the user's query receive a higher score.
- **Services and Cuisine Scoring**: Restaurants are further scored based on the presence of requested facilities and services, as well as the type of cuisine.
- **Sorting**: The results are sorted by the accumulated score in descending order using "heapq" for efficient sorting.

### Result
The final result is a sorted list of restaurants, with the most relevant ones appearing at the top, based on the combination of query match, TF-IDF score, price range, services, and cuisine type.

# Output
### User Query Input and Output

This is the section of the code where the user is prompted to input their query parameters. The program will then execute the customized_query function based on these inputs and produce the corresponding results.

1. **Query Text**  
   The user is asked to enter the query text (e.g., "restaurant with garden").

2. **Price Range**  
   The user provides a desired price range between 1 and 5, which corresponds to different pricing levels for the restaurants.

3. **Cuisine Types**  
   The user is prompted to enter a list of cuisine types separated by commas (e.g., "Ligurian, Italian"). This will be used to filter the restaurants based on cuisine.

4. **Services**  
   The user is asked to specify a list of services separated by commas (e.g., "Air conditioning, Car park"). This will help narrow down the search to restaurants offering the specified services.

After the user inputs these parameters, the customized_query function is called with the user's inputs along with other necessary data like the dictionary, inverted_index, TFiDF, and vocabulary variables. The result is then displayed, which will show restaurants that match the criteria specified by the user.

This cell is responsible for triggering the output based on the user's query.


In [50]:
query_text = input("Enter the query text (e.g., 'restaurant with garden'): ")
print("Enter the query text (e.g., 'restaurant with garden'): " +query_text)
query_price = int(input("Enter the desired price range (1-5): "))
print("Enter the desired price range (1-5): " +str(query_price))
query_cuisine = input("Enter the desired cuisines separated by commas (e.g., 'Ligurian, Italian'): ")
print("Enter the desired cuisines separated by commas (e.g., 'Ligurian, Italian'): " +query_cuisine)
query_services = input("Enter the desired services separated by commas (e.g., 'Air conditioning, Car park'): ")
print("Enter the desired services separated by commas (e.g., 'Air conditioning, Car park'): "+ query_services)

customized_query(query_text, dictionary, inverted_index, TFiDF, vocabulary, query_price, query_services, data = data)

Enter the query text (e.g., 'restaurant with garden'): restaurant with garden
Enter the desired price range (1-5): 3
Enter the desired cuisines separated by commas (e.g., 'Ligurian, Italian'): Italian
Enter the desired services separated by commas (e.g., 'Air conditioning, Car park'): Car Parking


,restaurantName,address,description,website,score
0,L'Asinello,"via Nuova 6, località Villa a Sesta, Castelnuo...",Situated at the entrance to a romantic village...,https://www.asinelloristorante.it/,3.93
1,La Masseria,"via Chiesa 61, località Marzaglia, Modena, 411...",What used to be an old watermill is now a welc...,https://www.ristorantemasseria.com/,3.21
2,Osteria Mood,"via Cesare Battisti 58, San Bartolomeo al Mare...",Situated just a few minutes from the seafront ...,https://osteriamood.it/,2.71
3,Il Papavero,"corso Garibaldi 112/113, Eboli, 84025, Italy","Papavero is the Italian for poppy, a plant whi...",https://www.ristoranteilpapavero.it/,2.71
4,Quartopiano,"via Chiabrera 34/c, Rimini, 47924, Italy",Perhaps because of its out - of - the - way lo...,https://www.quartopianoristorante.com/,2.71
...,...,...,...,...,...
178,L'Insolita Trattoria Tre Soldi,"via Gabriele d'Annunzio 4r/a, Florence, 50135,...",A long-established trattoria opened by a marri...,https://insolitatrattoria.it/,-1.00
179,Matteo Grandi in Basilica,"piazza dei Signori 1, Vicenza, 36100, Italy",Situated on the first floor of a beautiful bui...,https://www.ristorantematteograndi.it,-1.00
180,Spinechile,"contra' Pacche 2, località Tretto, Schio, 3601...",Situated in a magical location amid the wooded...,https://www.spinechile.com,-1.00
181,La Caravella dal 1959,"via Matteo Camera 12, Amalfi, 84011, Italy",La Caravella’s history is intricately linked t...,https://www.ristorantelacaravella.it/,-1.00


### Are the results you obtain better than with the previous scoring function? Explain and compare results.


In this scenario, we are comparing the results of three different search methods for the query "vegan restaurant with garden":

1. **The scoring function with weighted variables** (used in the `customized_query` function).
2. **The similarity score function** (used in `ranked_search`).
3. **The conjunctive query** that matches keywords in the description (used in `conjunctive_query`).

To evaluate which approach produces the "better" results, we need to consider **precision**, **relevance**, and **how well each method meets the intent of the query**. In this case, the query specifically asks for "vegan restaurants with garden," which requires results that are not only vegan but also likely have outdoor garden spaces. 

#### **1. Results from the Scoring Function with Weighted Variables:**
| **Restaurant Name** | **Address**                                   | **Description**                                                                                                           | **Website**                                                 | **Score** |
|----------------------|-----------------------------------------------|---------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------|-----------|
| Li Lioni            | SS 131 km 244,400 - regione Li Lioni, Porto Torres, Italy | The highlights of this restaurant just outside Porto Torres include rustic charm and flavorful Sardinian cuisine.         | [tenutalilioni.it](https://www.tenutalilioni.it/)           | 7.76      |
| Il Mirto            | via Provinciale Lacco, 284, Forio, 80075, Italy | When you arrive at the luxury Botania hotel, you'll find this serene and delightful vegetarian restaurant.                 | [ilmirtoristorantevegetariano.it](http://www.ilmirtoristorantevegetariano.it) | 6.76      |


**Analysis**: 
- This function ranks the results based on various factors, including TF-IDF, price, services, and the type of cuisine. **Li Lioni** stands out here as a restaurant with a garden, and it receives a high score (7.76), likely due to its relevance to the query.
- The results are well-rounded and consider multiple factors, such as the type of cuisine and the environment (likely considering the garden).
- **Il Mirto** despite being relevant in terms of cuisine, rank low due to other factors like description match and amenities.

#### **2. Results from the Similarity Score Function:**

| **restaurantName** | **address** | **description** | **website** | **SimilarityScore** |
|--------------------|-------------|-----------------|-------------|---------------------|
| Sale Grosso        | vicolo De' Facchini 4/a, Bologna, 40124, Italy | A simple, bistro-style restaurant serving we... | NaN | 0.264720 |
| Li Lioni           | SS 131 km 244,400 - regione Li Lioni, Porto To... | The highlights of this restaurant just outside... | [Link](https://www.tenutalilioni.it/) | 0.183576 |
| Il Mirto           | via Provinciale Lacco, 284, Forio, 80075, Italy | When you arrive at the luxury Botania hotel, y... | [Link](http://www.ilmirtoristorantevegetariano.it) | 0.157729 |

**Analysis**:
- The **similarity score function** ranks the results by the cosine similarity between the query vector and the document vectors. 
- **Sale Grosso** and **Li Lioni** appear in the top results, but they receive relatively low scores (0.2647 and 0.1836), suggesting that they are not particularly relevant to the query, even though they may contain some of the keywords. 
- **Il Mirto** also appears, but it is not particularly relevant either. This function doesn’t account for factors like garden spaces, services, or even cuisine in a meaningful way, relying solely on textual similarity, which often leads to less relevant results when the query is specific (like "vegan restaurant with garden").

#### **3. Results from the Conjunctive Query (Exact Keyword Match in Description):**

| **restaurantName** | **address** | **description** | **website** |
|--------------------|-------------|-----------------|-------------|
| Il Mirto           | via Provinciale Lacco, 284, Forio, 80075, Italy | When you arrive at the luxury Botania hotel, y... | [Link](http://www.ilmirtoristorantevegetariano.it) |
| Li Lioni           | SS 131 km 244,400 - regione Li Lioni, Porto To... | The highlights of this restaurant just outside... | [Link](https://www.tenutalilioni.it/) |

**Analysis**:
- The **conjunctive query** approach strictly matches keywords (in this case, "vegan" and "garden") in the description. 
- It returns **Il Mirto** and **Li Lioni**, which are vegan-friendly restaurants, but there is no clear mention of a garden or outdoor space in the descriptions provided.
- The results are less relevant in terms of matching the user's expectations of finding a vegan restaurant with a garden, as it doesn't consider context or additional factors like restaurant ambiance or setting.

### **Comparison and Conclusion:**

- **The scoring function with weighted variables** (used in customized_query) produces the most relevant and well-rounded results, as it integrates multiple factors that can affect a restaurant’s suitability for the query. The **Li Lioni** restaurant is a standout due to its garden, and the overall scores give a clearer picture of the restaurants' relevance. It is especially useful for a query that implies several considerations (vegan, garden, type of restaurant).
  
- **The similarity score function** (used in ranked_search) is less effective in this context. While it ranks **Sale Grosso** and **Li Lioni**, it doesn’t capture the nuances of the query well because it relies purely on textual similarity, without considering the full context of the query (e.g., garden space, vegan offerings). It thus produces results that are not well aligned with the user's search intent.

- **The conjunctive query** (used in conjunctive_query), while focusing on keyword matching, does not perform well either. It doesn’t fully capture the intent of the user, as it returns restaurants like **Il Mirto** and **Li Lioni**, which may be vegan but do not mention a garden in the description. This method is highly restrictive because it only matches exact keywords and does not account for context or other relevant criteria like ambiance or services.

**Conclusion**: 
The **scoring function with weighted variables** offers **the best results** because it provides a more comprehensive and accurate ranking of restaurants, taking into account not only the query keywords but also essential factors like the restaurant's environment (e.g., garden) and amenities, alongside cuisine type and price. This makes it more likely to return results that align with the user's needs, whereas the other two functions either rely too heavily on simple keyword matching or similarity without context, leading to less precise outcomes.



# 4. Visualizing the Most Relevant Restaurants

Maps can provide users with an easy way to see where restaurants are located. This is especially useful for understanding which regions in Italy have more options.

Steps for Visualization:
1. Geocode Locations: Collect information on unique restaurant locations in Italy (in the format of City and Region). You can use tools such as Google API, OpenStreetMap, or a pre-defined list to retrieve representative coordinates for each region.
2. Ask a Large Language Model (LLM): Alternatively, you can compile a list of unique cities and regions in Italy, formatted as (City, Region), and ask an LLM (e.g., ChatGPT) to provide coordinates for these locations. This can be an efficient way to gather data without using API calls. Just make sure that the retrieved information is correct and helpful.
3. Map Setup: Use a mapping library like plotly or folium to create a visual display of restaurants by region.
4. Encoding Price Ranges: Incorporate a visual representation for price ranges:
    * Use color-coding or marker size to represent the restaurant's price range (€, €€, €€€, €€€€).
    * Include a legend for interpreting price levels.
5. Plot Top-K Restaurants: Use the custom score from Step 3 to select the top-k restaurants for display.
This map will give users an overview of restaurant options across different regions in Italy, with an indication of cost based on visual cues.

---

First, we will extract the cities from the addresses of the restaurants using the `get_city` function defined in the `functions.py` file. 


In [27]:
# Make a column for the city of the restaurant
data["city"] = data.address.apply(get_city)
# Get the unique cities in the dataset
unique_cities = list(data.city.unique())
print(f"Number of Unique cities in the dataset: {len(unique_cities)}")


Number of Unique cities in the dataset: 1160


##### Section 4.1:
* Geocode Locations: Collect information on unique restaurant locations in Italy (in the format of City and Region). You can use tools such as Google API, OpenStreetMap, or a pre-defined list to retrieve representative coordinates for each region.

 We will use the unique cities extracted above to get their corresponding *regions* and the values of the *latitude* and *longitude* of each region. This is required by the exercise.

 We will make use of the two functions defined in the `functions.py` file to get the regions, their latitude and longitude, and eventually latitude and longitude of also each individual address of the restaurant in the dataset. The functions are:

 * `get_region_and_coordinates`: This function takes in the unique cities and an API KEY and returns the region and the latitude and longitude of the region.
 * `get_coordinates`: This function takes in the address of the restaurant and an API KEY and returns the latitude and longitude of the address of the restaurant in question.

 Both of these functions are defined in the functions.py file and are well documented.



In [ ]:
API_KEY = 'YOUR API KEY'
with open('city_region_coordinates.csv', 'a') as f:
    for city in tqdm(unique_cities, desc="Fetching coordinates for regions"):
        region, lat, lng = get_region_and_coordinates(city, API_KEY)
        if region and lat and lng:
            f.write(f"{city},{region},{lat},{lng}\n")


with open('restaurant_coordinates.csv', 'a') as f:
    addresses = list(data.address)
    for address in tqdm(addresses, desc="Fetching coordinates for restaurants"):
        lat, lng = get_coordinates(address, API_KEY)
        if lat and lng:
            f.write(f"{lat},{lng}\n")

Having the coordinates of the regions and the restaurants stored in two different files, namely:
- `city_region_coordinates.csv`
- `restaurant_coordinates.csv

We can now load them, do some preprocessing (like color mapping the regions and merging the two datasets), and visualize them on a map using the folium library.

In [28]:
import matplotlib.pyplot as plt
# Read the data from the csv file containing the coordinates of the regions
region_data = pd.read_csv('Files/city_region_coordinates.csv', names=['city', 'region', 'lat_region', 'lng_region'])

# calculate the number of regions in Italy. This will be useful for coloring the regions in the map 
num_regions = len(region_data['region'].unique())
print(f"Number of regions in Italy: {num_regions}")


# Assign colors to regions
colors = plt.get_cmap('tab20', num_regions)
# Create a dictionary that maps each region to a color
region_color_map = {region: colors(i) for i, region in enumerate(region_data['region'].unique())}
# Map the colors to the regions
region_data['color_region'] = region_data['region'].map(region_color_map)

# Convert RGBA to hex for folium
region_data['color_region'] = region_data['color_region'].apply(lambda x: '#%02x%02x%02x' % (int(x[0]*255), int(x[1]*255), int(x[2]*255)))

region_data.head()

Number of regions in Italy: 27


,city,region,lat_region,lng_region,color_region
0,Melizzano,Campania,41.112508,14.845462,#1f77b4
1,Sant'Omobono Terme,Lombardy,45.479067,9.845243,#1f77b4
2,Furore,Campania,41.112508,14.845462,#1f77b4
3,Santa Marina Salina,Sicily,37.397930,14.658782,#aec7e8
4,Rimini,Emilia-Romagna,44.596761,11.218640,#ff7f0e


The table above shows the first 5 rows of the region_data dataframe. The dataframe contains the city, region, latitude, longitude, and the color of each region. The color is used to color the regions in the map.

Next, we will load the data containing the coordinates of the restaurants. We will then merge the data with the region_data dataframe to get the region of each restaurant. This will be useful for using just a single dataframe to plot the regions and the restaurants on the map.



In [29]:
print(data.shape)
coordinates_data = pd.read_csv('Files/restaurant_coordinates.csv', names=['address_lat', 'address_lng'])

print(coordinates_data.shape)

(2029, 15)
(2028, 2)


There is a mismatch in the number of rows between the data and the coordinates data. This is because one restaurantm may have inserted wrong address which the Google Api Maps could not find and it subsequently did not end up in our restaurant_coordinates file. After debugging of hours, I found that the restaurant at index 1176 has a faulty address. I will remove this restaurant from the data. 

In [30]:
data.iloc[1176]

restaurantName                                           Albergo Al Sole
address                 Via Collegio, 33, VAT #03581110263, Asolo, Italy
city                                                    VAT #03581110263
postalCode                                                           NaN
country                                                            Italy
priceRange                                                           NaN
cuisineType                                 Traditional Elegance & Quiet
description            Even for Asolo, where old stone villas seem to...
facilitiesServices                                                      
creditCards                                                          NaN
phoneNumber                                                          NaN
website                                                              NaN
cleaned_description    [even, asolo, old, stone, villa, seem, sprout,...
SimilarityScore                                    

In [31]:
data.drop(data.index[1176], inplace=True)

In [32]:
# Reset the index
data.index = range(len(data))

In [33]:
print(coordinates_data.iloc[1177]) # print the coordinates of the restaurant from the row after the deleted row
data.iloc[1177]  # print the restaurant information from the row after the deleted row

address_lat    45.803268
address_lng    11.914019
Name: 1177, dtype: float64


restaurantName                                               La Terrazza
address                             via Collegio 33, Asolo, 31011, Italy
city                                                               Asolo
postalCode                                                       31011.0
country                                                            Italy
priceRange                                                            €€
cuisineType                                               Modern Cuisine
description            The aptly named Terrazza is a delightful open ...
facilitiesServices                   Air conditioning; Car park; Terrace
creditCards                                       amex; mastercard; visa
phoneNumber                                              +39 0423 951332
website                                  https://albergoalsoleasolo.com/
cleaned_description    [aptli, name, terrazza, delight, open, air, re...
SimilarityScore                                    

Having inserted the coordinates into the google maps do show the location of La Terrazza on the map, confirming that the coordinates to address mapping are correct.

In [34]:
print(data.shape)
print(coordinates_data.shape)

(2028, 15)
(2028, 2)


In [35]:
# merge the data containing the restaurants information with the data containing the coordinates of the restaurants
data = pd.merge(data, coordinates_data, left_index=True, right_index=True)
data.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,cleaned_description,SimilarityScore,id,address_lat,address_lng
0,Locanda Radici,"SP 21, contrada San Vincenzo, Melizzano, 82030...",Melizzano,82030.0,Italy,€€,"Modern Cuisine, Campanian",A rustic restaurant with contemporary Mediterr...,Air conditioning; Car park; Garden or park; Gr...,amex; mastercard; visa,+39 0824 944506,https://www.locandaradici.it/,"[rustic, restaur, contemporari, mediterranean,...",0.013850,0,41.234823,14.476782
1,Posta,"viale Vittorio Veneto 169, Sant'Omobono Terme,...",Sant'Omobono Terme,24038.0,Italy,€€€,Italian,"Situated in the Imagna valley, this welcoming,...",Air conditioning; Wheelchair access,amex; dinersclub; mastercard; visa,+39 035 851134,https://www.frosioristoranti.it,"[situat, imagna, valley, welcom, famili, run, ...",0.004256,1,45.814752,9.532718
2,Hostaria Baccofurore,"via G.B. Lama 9, Furore, 84010, Italy",Furore,84010.0,Italy,€€,"Regional Cuisine, Farm to table",Patience is needed to get to this restaurant f...,Air conditioning; Car park; Garden or park; Gr...,amex; dinersclub; jcb; maestrocard; mastercard...,+39 089 830360,https://www.baccofurore.it/,"[patienc, need, get, restaur, coast, result, w...",0.008170,2,40.618573,14.550961
3,Nni Lausta,"via Risorgimento 188, Santa Marina Salina, 980...",Santa Marina Salina,98050.0,Italy,€€,"Seafood, Seasonal Cuisine","Fish plays the starring role here, where the a...",Terrace,amex; jcb; maestrocard; mastercard; visa,+39 090 984 3486,NaN,"[fish, play, star, role, authent, tradit, flav...",0.071051,3,38.559471,14.871372
4,Osteria de Börg,"via Forzieri 12, Rimini, 47921, Italy",Rimini,47921.0,Italy,€,"Cuisine from Romagna, Traditional Cuisine",Borgo San Giuliano is situated just a stroll f...,Terrace,amex; maestrocard; mastercard; visa,+39 0541 56074,https://www.osteriadeborg.it/,"[borgo, san, giuliano, situat, stroll, sea, ol...",0.003738,4,44.064796,12.563341


In [36]:
data.isna().sum()

restaurantName           0
address                  0
city                     0
postalCode              45
country                  0
priceRange              45
cuisineType              0
description              0
facilitiesServices       0
creditCards             49
phoneNumber             45
website                153
cleaned_description      0
SimilarityScore          0
id                       0
address_lat              0
address_lng              0
dtype: int64

In [37]:
# dropping the rows with missing values in the column priceRange because we want to use this column to represent colors on maps later
data.dropna(subset=['priceRange'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [38]:
# now, encode for price for each restaurant
price_colors = {
    '€': 'green',
    '€€': 'yellow',
    '€€€': 'orange',
    '€€€€': 'red'
}
# Assign colors to price ranges
data['color'] = data['priceRange'].map(price_colors)

data.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,cleaned_description,SimilarityScore,id,address_lat,address_lng,color
0,Locanda Radici,"SP 21, contrada San Vincenzo, Melizzano, 82030...",Melizzano,82030.0,Italy,€€,"Modern Cuisine, Campanian",A rustic restaurant with contemporary Mediterr...,Air conditioning; Car park; Garden or park; Gr...,amex; mastercard; visa,+39 0824 944506,https://www.locandaradici.it/,"[rustic, restaur, contemporari, mediterranean,...",0.013850,0,41.234823,14.476782,yellow
1,Posta,"viale Vittorio Veneto 169, Sant'Omobono Terme,...",Sant'Omobono Terme,24038.0,Italy,€€€,Italian,"Situated in the Imagna valley, this welcoming,...",Air conditioning; Wheelchair access,amex; dinersclub; mastercard; visa,+39 035 851134,https://www.frosioristoranti.it,"[situat, imagna, valley, welcom, famili, run, ...",0.004256,1,45.814752,9.532718,orange
2,Hostaria Baccofurore,"via G.B. Lama 9, Furore, 84010, Italy",Furore,84010.0,Italy,€€,"Regional Cuisine, Farm to table",Patience is needed to get to this restaurant f...,Air conditioning; Car park; Garden or park; Gr...,amex; dinersclub; jcb; maestrocard; mastercard...,+39 089 830360,https://www.baccofurore.it/,"[patienc, need, get, restaur, coast, result, w...",0.008170,2,40.618573,14.550961,yellow
3,Nni Lausta,"via Risorgimento 188, Santa Marina Salina, 980...",Santa Marina Salina,98050.0,Italy,€€,"Seafood, Seasonal Cuisine","Fish plays the starring role here, where the a...",Terrace,amex; jcb; maestrocard; mastercard; visa,+39 090 984 3486,NaN,"[fish, play, star, role, authent, tradit, flav...",0.071051,3,38.559471,14.871372,yellow
4,Osteria de Börg,"via Forzieri 12, Rimini, 47921, Italy",Rimini,47921.0,Italy,€,"Cuisine from Romagna, Traditional Cuisine",Borgo San Giuliano is situated just a stroll f...,Terrace,amex; maestrocard; mastercard; visa,+39 0541 56074,https://www.osteriadeborg.it/,"[borgo, san, giuliano, situat, stroll, sea, ol...",0.003738,4,44.064796,12.563341,green


Now that we have the dataset ready, we can plot the map using the `make_map` function which will create an interactive map using folium. The make_map function is defined in the functions.py file and is well commented for better understanding.

In [39]:
map_filename = 'michelin_restaurants_map.html'
make_map(data, region_data, map_filename)

/Users/saifdev/Desktop/ADM/ADM_HW_3/scripts/functions.py:722: UserWarning: color argument of Icon should be one of: {'pink', 'lightgray', 'black', 'darkpurple', 'green', 'lightgreen', 'darkgreen', 'red', 'darkred', 'darkblue', 'lightblue', 'purple', 'cadetblue', 'lightred', 'beige', 'white', 'gray', 'blue', 'orange'}.
  icon=folium.Icon(color=row["color"], icon="cutlery", prefix="fa")


Map saved to michelin_restaurants_map.html


**To view the map, click [here](https://nbviewer.org/github/saifali03/ADM_HW3/blob/main/michelin_restaurants_map.html)**

To perform task 5, We will first load the results from the search engine defined based on our scoring criteria. Having these results (data containing information about the restaurants from the search engine), we will merge them with the unified dataset we made earlier. There will be some duplicate columns in this new merged dataset, so we will drop them. We will then make a map using the make_map function and save it as a html file. The map will contain the restaurants that are the most relevant to the query entered by the user. The restaurants will be colored based on their price range and the regions will be colored based on the region they belong to. The map will be saved as a html file and displayed in the browser.

In [48]:
engine_results = customized_query(query_text, dictionary, inverted_index, TFiDF, vocabulary, query_price, query_services, data = data)
top_10_data = engine_results.head(10)
top_10_data

,restaurantName,address,description,website,score
0,L'Asinello,"via Nuova 6, località Villa a Sesta, Castelnuo...",Situated at the entrance to a romantic village...,https://www.asinelloristorante.it/,3.93
1,La Masseria,"via Chiesa 61, località Marzaglia, Modena, 411...",What used to be an old watermill is now a welc...,https://www.ristorantemasseria.com/,3.21
2,Osteria Mood,"via Cesare Battisti 58, San Bartolomeo al Mare...",Situated just a few minutes from the seafront ...,https://osteriamood.it/,2.71
3,Il Papavero,"corso Garibaldi 112/113, Eboli, 84025, Italy","Papavero is the Italian for poppy, a plant whi...",https://www.ristoranteilpapavero.it/,2.71
4,Quartopiano,"via Chiabrera 34/c, Rimini, 47924, Italy",Perhaps because of its out - of - the - way lo...,https://www.quartopianoristorante.com/,2.71
5,Antica Osteria Magenes,"via Cavour 7, Barate, 20083, Italy",This fascinating restaurant occupying an old h...,https://osteriamagenes.com/,1.71
6,Gardenia,"corso Torino 9, Caluso, 10014, Italy","Even if you’re not a local, you’ll feel at hom...",https://www.gardeniacaluso.com/,1.71
7,Lupaia,"Località Lupaia 74, Torrita di Siena, 53049, I...",Situated in a charming Tuscan hamlet nestled i...,https://www.lupaia.com,1.71
8,Sarri,"lungomare C. Colombo 108, Borgo Prino, Imperia...",This attractive restaurant boasts a beautiful ...,https://www.ristorantesarri.it/,1.71
9,Laurin,"via Laurin 4, Bolzano, 39100, Italy",In the magnificent setting of the exclusive Pa...,http://www.laurin.it,1.71


In [53]:
data.shape

(1983, 18)

In [54]:
restaurant_data = pd.DataFrame()
for restaurant in top_10_data["restaurantName"]:
    if restaurant in data["restaurantName"].values:
        # extract those restaurants from the dataset
        temp_data = data[data["restaurantName"] == restaurant].copy()
        temp_data["score"] = top_10_data[top_10_data["restaurantName"] == restaurant]["score"].values[0]
        restaurant_data = pd.concat([restaurant_data, temp_data])
top_10_data = restaurant_data.copy()
top_10_data

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website,cleaned_description,SimilarityScore,id,address_lat,address_lng,color,score
109,L'Asinello,"via Nuova 6, località Villa a Sesta, Castelnuo...",Castelnuovo Berardenga,53019.0,Italy,€€€,"Tuscan, Creative",Situated at the entrance to a romantic village...,Air conditioning; Garden or park; Restaurant o...,mastercard; visa,+39 0577 359279,https://www.asinelloristorante.it/,"[situat, entranc, romant, villag, chianti, reg...",0.062225,116,43.385086,11.480640,orange,3.93
51,La Masseria,"via Chiesa 61, località Marzaglia, Modena, 411...",Modena,41123.0,Italy,€,Apulian,What used to be an old watermill is now a welc...,Car park; Terrace,amex; mastercard; visa,+39 059 389262,https://www.ristorantemasseria.com/,"[use, old, watermil, welcom, restaur, rustic, ...",0.007013,55,44.652603,10.898632,green,3.21
102,Osteria Mood,"via Cesare Battisti 58, San Bartolomeo al Mare...",San Bartolomeo al Mare,18016.0,Italy,€€,"Ligurian, Modern Cuisine",Situated just a few minutes from the seafront ...,Car park; Terrace,mastercard; visa,+39 333 998 0201,https://osteriamood.it/,"[situat, minut, seafront, promenad, quiet, res...",0.005056,107,43.923426,8.096208,yellow,2.71
296,Il Papavero,"corso Garibaldi 112/113, Eboli, 84025, Italy",Eboli,84025.0,Italy,€€,"Modern Cuisine, Campanian","Papavero is the Italian for poppy, a plant whi...",Air conditioning; Terrace,mastercard; visa,+39 0828 330689,https://www.ristoranteilpapavero.it/,"[papavero, italian, poppi, plant, grow, tall, ...",0.068046,307,40.618982,15.054047,yellow,2.71
1245,Quartopiano,"via Chiabrera 34/c, Rimini, 47924, Italy",Rimini,47924.0,Italy,€€,"Creative, Seafood",Perhaps because of its out - of - the - way lo...,Air conditioning; Car park; Interesting wine l...,amex; dinersclub; maestrocard; mastercard; visa,+39 0541 393238,https://www.quartopianoristorante.com/,"[perhap, way, locat, busi, district, would, ne...",0.010358,1279,44.047454,12.594462,yellow,2.71
367,Antica Osteria Magenes,"via Cavour 7, Barate, 20083, Italy",Barate,20083.0,Italy,€€€,"Italian, Modern Cuisine",This fascinating restaurant occupying an old h...,Air conditioning; Garden or park; Terrace; Whe...,amex; mastercard; visa,+39 02 908 5125,https://osteriamagenes.com/,"[fascin, restaur, occupi, old, hous, renov, mo...",0.093846,378,45.384862,9.039808,orange,1.71
993,Gardenia,"corso Torino 9, Caluso, 10014, Italy",Caluso,10014.0,Italy,€€€,"Piedmontese, Modern Cuisine","Even if you’re not a local, you’ll feel at hom...",Air conditioning; Car park; Interesting wine l...,amex; mastercard; visa,+39 011 983 2249,https://www.gardeniacaluso.com/,"[even, local, feel, home, old, properti, offer...",0.005471,1022,45.303382,7.890069,orange,1.71
1071,Lupaia,"Località Lupaia 74, Torrita di Siena, 53049, I...",Torrita di Siena,53049.0,Italy,€€€,"Country cooking, Farm to table",Situated in a charming Tuscan hamlet nestled i...,Car park; Garden or park; Terrace,amex; maestrocard; mastercard; visa,+39 0577 191 7066,https://www.lupaia.com,"[situat, charm, tuscan, hamlet, nestl, hill, m...",0.005498,1103,43.112922,11.750263,orange,1.71
1189,Sarri,"lungomare C. Colombo 108, Borgo Prino, Imperia...",Imperia,18100.0,Italy,€€€,"Seafood, Modern Cuisine",This attractive restaurant boasts a beautiful ...,Air conditioning; Interesting wine list; Terra...,amex; mastercard; visa,+39 0183 754056,https://www.ristorantesarri.it/,"[attract, restaur, boast, beauti, locat, charm...",0.002972,1223,43.870194,8.005792,orange,1.71
1921,Laurin,"via Laurin 4, Bolzano, 39100, Italy",Bolzano,39100.0,Italy,€€€,"Modern Cuisine, Classic Cuisine",In the magnificent setting of the exclusive Pa...,Air conditioning; Garden or park; Terrace,amex; mastercard; visa,+39 0471 311000,http://www.laurin.it,"[magnific, set, exclus, parkhotel, laurin, res...",0.004046,1966,46.497836,11.355942,orange,1.71


In [55]:
map_filename = 'my_engine_results_map.html'
make_map(dataset = top_10_data, region_data= region_data, output_filename= map_filename)

/Users/saifdev/Desktop/ADM/ADM_HW_3/scripts/functions.py:722: UserWarning: color argument of Icon should be one of: {'pink', 'lightgray', 'black', 'darkpurple', 'green', 'lightgreen', 'darkgreen', 'red', 'darkred', 'darkblue', 'lightblue', 'purple', 'cadetblue', 'lightred', 'beige', 'white', 'gray', 'blue', 'orange'}.
  icon=folium.Icon(color=row["color"], icon="cutlery", prefix="fa")


Map saved to my_engine_results_map.html


**To view the map, click [here](https://nbviewer.org/github/saifali03/ADM_HW3/blob/main/my_engine_results_map.html)**

## Algoritmic Question (AQ)

A robot is tasked with collecting $n$ packages located at coordinates $(x_i, y_i)$ starting at position $(0,0)$, where:
​
+ $t$ is the number of test cases
+ $n$ is the number of packages
+ $(x_i, y_i)$ are the coordinates of the $i$-th package.


The robot can only move right (R) or up (U). Therefore, to successfully collect all packages without retracing its steps (moving down or left), the following condition must hold for all pairs $i<j$ in the list of package coordinates:  $$x_i\leq x_j \iff  y_i\leq y_j$$

This ensures that the robot can reach every package while moving strictly in the allowed directions.

### **Input**

The first line contains t — the number of test cases.
Each test case starts with n, the number of packages. The next n lines contain the coordinates $(x_i,y_i)$ for each package $i\in\{1,...,n\}$.

### **Output**
For each test case, print "YES" and the lexicographically smallest path, or "NO" if it’s impossible to collect all packages.

### **Pseudo-algorithm**

`t` = input number of test cases  

**For** each test case:  
- get `n` = input number of packages  
- Read and store package coordinates in `coords`  

  **If** `n == 1`:  
  - Extract `(x, y)` from `coords`  
  - Print "YES" and path = `'R' * x + 'U' * y`  
  - **Skip** the rest of the test case  

  **Else**:  
  - Sort `coords` lexicographically  
  - *Initialize* `possible = True`  

  **For** `(x1, y1), (x2, y2)` in consecutive pairs of `coords`:  
  - **If** `x2 < x1` or `y2 < y1`:  
    - `possible = False` 

  **If** not `possible`:  
  - Print "NO"  

  **Else**:  
  - *Initialize* `(x0, y0) = (0, 0)` and `path = ""`  
  - **For** `(x, y)` in `coords`:  
    - Add `'R' * (x - x0)` and `'U' * (y - y0)` to `path`  
    - Update `(x0, y0) = (x, y)`  
  - Print "YES" and `path`  

It follows the implementation of the pseudocode for the algorithm

### **AQ1. Algorithm**



In [ ]:
t = int(input('test cases'))

for test in range(t):
    n = int(input('number of packages'))
    coords = []
    for i in range(n):
        (x,y) = map(int, input(f'Coordinates of {i+1}-th package').split())
        coords.append((x,y))

    #if the condition doesn't hold for any pair, print no
    if n == 1:  # Special case: only one package
        x, y = coords[0]
        print("YES")
        print('R' * x + 'U' * y)
        continue

    else: 
        sorted_pcks= sorted(coords)
        possible = True

        for (x1, y1), (x2, y2) in zip(sorted_pcks, sorted_pcks[1:]):
            if x2 < x1 or y2 < y1:
                possible = False
                break

        if not possible:
            print('NO')
        else:
            path = []
            #generate lexicographically shortest path:
            tempx, tempy = 0,0 #starting at zero
            for x,y in sorted_pcks:
                steps_r = x-tempx
                steps_u = y-tempy
                path.append(steps_r * 'R')
                path.append(steps_u * 'U')
                tempx, tempy =  x, y

            print('Yes')
            print(''.join(path))


Yes
RUUURRRRUU
NO
YES
RRRRUUU


### **AQ2. Proof of Correctness**
##### Case 1: n = 1
For a single package $(x, y)$, the robot can always reach the package using exactly x right moves and y up moves.
The output path $'R' ⋅ x + 'U' ⋅ y$ is valid and lexicographically smallest by construction.
##### Case 2: General Case (n > 1)
Sorting ensures that the robot processes the packages in order of increasing $x$-coordinates. Within packages with the same $x$-coordinate, it processes those with increasing $y$-coordinates.
If the sorted order is invalid (i.e., $x_2 < x_1$ or $y_2 < y_1$), the algorithm correctly identifies this and outputs "NO."
##### Feasibility:
The algorithm checks all pairs of consecutive packages in the sorted list to ensure that moving between them is feasible without moving left or down.
If any pair violates this condition, the robot cannot collect all packages, and the algorithm outputs "NO."
##### Path Construction:
The algorithm constructs the path by calculating the exact number of right and up moves needed to reach each package from the current position.
Because the coordinates are sorted lexicographically, the robot always moves in the lexicographically smallest way.
By appending right moves first ('R' ⋅ steps_r) followed by up moves ('U' ⋅ steps_u), the algorithm ensures the path is constructed in lexicographical order.
##### Output Correctness:
If the feasibility check passes, the algorithm outputs "YES" and the constructed path. Both are correct because the robot can visit all packages in order without violating the movement constraints.

We can further consider some edge cases:
+ Single Test Case (t == 1): The loop correctly handles a single test case.
+ Multiple Test Cases: The algorithm handles each test case independently, so results from one test case do not affect others.

Boundary Values:
+ If n == 1, the algorithm outputs the correct path.
+ If coords contain only points along the x-axis or y-axis, the algorithm handles these correctly.
+ If any package is unreachable (due to invalid coordinates), the algorithm outputs "NO."

### **AQ.3 Time Complexity**

The **outer loop** requires $t$ iterations, one for each test case, so it takes $O(t)$. The **inner loop** requires $n$ iterations, one for each different number of packages, in order to read and append the coordinates, which are operations that require constant time, hence of the order $O(n)$.

Inside the loop, in the best case, that is $n = 1$, we just compute this check, write the path which has lenght equal to the sum of the coordinates, print it and exit the inner loop to go to the next test case, which takes constant time (const).

In the worst case, that is $n>1$, after reading the $n$ coordinates and append them to the $n$-dimentional list $(const⋅n)$, we **sort it**, wich costs $O(nlogn)$, since python uses the TimSort algorithm.

Then, for each of the $n-1$ pairs of coordinates in the $n$-dimentional sorted list we need to compare their $x$ and $y$ to check the feasibility of the operation, performing a total of $2⋅(n-1)$ checks. Subsequentially we iterate once again over the coordinates of the list to compute the number of 'R' and 'U' steps, and perform constant time operations, which costs $O(n)$.

Finally we have $O(t⋅[n+const+nlogn+2(n-1)+n(const)])$, but since the term $nlogn$ clearly dominates on the others it is summed with, we end up with $$T(t,n) = O(t⋅nlogn)$$


### **AQ.4 LLM tool complexity evaluation**

We instructed ChatGPT with the following prompt: 

*"Evaluate the time complexity of the following code using Big O notation"* , and fed it with our algorithm.
#### ChatGPT Response:
" To evaluate the time complexity of the given code, we break it down into its major components:

1. Outer Loop (Test Cases):
    This loop runs for $t$ test cases, where $t$ is the number of test cases provided as input.
2. Inner Loop (Reading Coordinates):
    This loop runs $n$ times for each test case, where $n$ is the number of packages in the test case.
    Time complexity: $O(n)$ for input and appending the coordinates to the coords list.
3. Special Case for $n == 1$:
    For the special case where $n == 1$, the algorithm directly calculates and prints the path. This involves creating strings of length $x$ and $y$, where
    $x$ and $y$ are the coordinates of the package.
    Time complexity for this case: $O(x+y)$
4. Sorting Coordinates:
    Sorting $n$ coordinates lexicographically has a time complexity of $O(nlogn)$.
5. Feasibility Check:
    The loop iterates through $n - 1$ pairs of coordinates to check the feasibility condition.
    Time complexity: $O(n)$.
6. Path Construction:
    The loop iterates over $n$ coordinates, and for each coordinate, it computes the number of steps (steps_r and steps_u) and appends the corresponding characters to the path.
    Appending strings and joining the path involve processing a total of $O(total path length)$, where the path length is proportional to the sum of the $x$ and $y$ coordinates of all packages: $\sum (x_i+y_i)$.
    Time complexity: $O(n+total path length)$.


7. Overall Time Complexity
    For each test case:
    + Input Reading: $O(n)$,
    + Special Case ($n == 1$): $O(x+y)$ (only if $n==1$).
    + Sorting: $O(nlogn)$
    + Feasibility Check: $O(n)$
    + Path Construction: $O$($n$+total path length).

    Combining these:

    Worst-case complexity per test case: $O(nlogn+n+total path length).$

For $t$ test cases:

Overall complexity: $O(t⋅(nlogn+n+total path length))$.
If we assume the path length scales linearly with $n$ (e.g. total path length = $O(n)$), the final complexity simplifies to:
$O(t⋅nlogn)$"

  The response provided is correct.

### **AQ.5 Greedy alorithm optimality**
> Assume now that the robot can also move towards the left or downwards, and consider the greedy approach: from the current location go to the closest package. Notice that now we can always collect all packages. Prove that the greedy algorithm is optimal (i.e., it minimizes the total distance traveled), or provide a counterexample showing that it is not.

In [ ]:
t = int(input("test cases"))

for test in range(t):
    n = int(input("number of packages"))
    coords = []
    for i in range(n):
        x, y = map(int, input(f"Coordinates of {i+1}-th package: ").split())
        coords.append((x, y))

    # Start at (0, 0)
    x0, y0 = 0, 0
    path = []
    
    while coords:
        # Find the nearest package based on Manhattan distance
        coords.sort(key=lambda p: abs(p[0] - x0) + abs(p[1] - y0))
        x, y = coords.pop(0)  # Get the nearest package
        
        # path instructions
        if x > x0:
            path.append('R' * (x - x0))
        elif x < x0:
            path.append('L' * (x0 - x))
        
        if y > y0:
            path.append('U' * (y - y0))
        elif y < y0:
            path.append('D' * (y0 - y))
        
        # Update current position
        x0, y0 = x, y

    print("Yes") #always possible
    print("".join(path))


Yes
RRRURRUULLLUUU
Yes
RRRRRRUUULLUUUUU


Sorting the packages by Manhattan distance has a time complexity of $O(n^2)$ in total across $n$ steps.
The path construction is an $O(total path length)$.
The overall complexity is then $O(t⋅n ^2)$, which is acceptable for small values of $n$, but for larger inputs, optimizations (e.g., using a priority queue) would be necessary.

The greedy algorithm described, where the robot moves to the closest package based on Manhattan distance, is not guaranteed to be optimal in terms of minimizing the total distance traveled. The robot here always chooses the nearest package (in terms of Manhattan distance) from the current location, without considering how this decision might affect future moves. This approach does not always lead to the globally optimal solution. The problem arises because choosing the "locally optimum" among the packages at every step can result in a suboptimal path when considering all packages.

#### Counterexample:
>Start: (0, 0)
>Packages at (2, 2), (5, 0), and (0, 5).

- Greedy Algorithm
    * Start at (0, 0).
    * Closest package: (2, 2) (distance = 4). Move to (2, 2).
    * Closest package from (2,2): (0, 5) and (5, 0) equally distant, so assume it chooses (5, 0) (distance = 5). Move to (5, 0).
    * Last package: (0, 5) (distance = 10).
    * Total distance (Greedy): $4+5+10=19$

- Possible Alternative Optimal Solution
    * Start at (0, 0).
    * Move to (0, 5) first (distance = 5).
    * From (0, 5) move to (2, 2) (distance = 5).
    * Finally, move to (5, 0) (distance = 5).
    * Total distance (Optimal): $5+5+5=15$

The greedy algorithm makes a locally optimal decision at each step but does not minimize the total distance. This counterexample proves that the greedy approach is not guaranteed to be optimal.